In [1]:
import pandas as pd
df_rentalcars = pd.read_csv('../../datasets/LetalCarOfContractType.csv')
df_rentalcars.head(3)

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [3]:
df_rentalcars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51304 entries, 0 to 51303
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 51304 non-null  int64  
 1   type_of_contract   51300 non-null  object 
 2   type_of_contract2  51303 non-null  object 
 3   channel            51304 non-null  object 
 4   datetime           51304 non-null  object 
 5   Term               51304 non-null  int64  
 6   payment_type       51304 non-null  object 
 7   product            51303 non-null  object 
 8   amount             51304 non-null  int64  
 9   state              51304 non-null  object 
 10  overdue_count      51304 non-null  int64  
 11  overdue            51302 non-null  object 
 12  credit rating      42521 non-null  float64
 13  bank               48544 non-null  object 
 14  cancellation       51279 non-null  object 
 15  age                40509 non-null  float64
 16  Mileage            405

## 범주형 / 연속형 : 범주형 따른 관계 확인
- 연속형이 비정규 분포이며, 범주형 항목이 2개일 때

### 예비 가설
- 분석 내용 : 채널 중 서비스 방문과 홈쇼핑 항목에 따른 월 렌탈 비용 평균이 차이가 있는가 ?
    + 귀무 가설 : 비용에 차이가 없다
    + 대립 가설 : 비용에 차이가 있다
- 결론 도출 : 대립 가설 성립

In [26]:
# 정규분포 여부 확인 : channel - 서비스 방문, 홈쇼핑/방송
conditions = 'channel in ("서비스 방문", "홈쇼핑/방송")'
df_rentalcars_channel_two = df_rentalcars.query(conditions)
len(df_rentalcars_channel_two)

28163

In [28]:
import scipy.stats as stats
results = stats.anderson(df_rentalcars_channel_two['amount']) # 정규분포
results

AndersonResult(statistic=1292.5765966678482, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=94187.21940134218, scale=10213.602310959697)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [5]:
df_rentalcars['channel'].value_counts().iloc[:2]

channel
서비스 방문    15186
홈쇼핑/방송    12977
Name: count, dtype: int64

In [15]:
conditions = 'channel == "서비스 방문"'
df_rentalcars_channel_first = df_rentalcars.query(conditions)
len(df_rentalcars_channel_first)

15186

In [11]:
conditions = 'channel == "홈쇼핑/방송"'
df_rentalcars_channel_second = df_rentalcars.query(conditions)
len(df_rentalcars_channel_second)

12977

## 연속형이 비정규 분포이며, 범주형 항목이 2개일 때
- Wilcoxon Test

In [12]:
import scipy.stats as stats

In [22]:
stats.ranksums(df_rentalcars_channel_first['amount'], df_rentalcars_channel_second['amount'])

RanksumsResult(statistic=74.731078009603, pvalue=0.0)

## 연속형이 비정규 분포이며, 범주형 항목이 3개 이상

In [18]:
df_rentalcars['channel'].value_counts().iloc[:3]

channel
서비스 방문    15186
홈쇼핑/방송    12977
렌탈총판       5544
Name: count, dtype: int64

In [19]:
conditions = 'channel == "렌탈총판"'
df_rentalcars_channel_third = df_rentalcars.query(conditions)
len(df_rentalcars_channel_third)

5544

In [24]:
test_list = [df_rentalcars_channel_first['amount']
             , df_rentalcars_channel_second['amount']
             ,df_rentalcars_channel_third['amount']]
stats.kruskal(*test_list)

KruskalResult(statistic=8232.430933915304, pvalue=0.0)